<a href="https://colab.research.google.com/github/AkarshMSonni/fetch/blob/main/Fetch_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q1. Inspect the unstructured JSON data and draw an ER Diagram

Converting unstructured data to a dataframe to review

In [42]:
#Importing libraries
import pandas as pd
import json
from ast import literal_eval


Normalizing and reading the Receipts table

In [29]:
# Read the JSON file with one JSON object per line
with open('/content/drive/MyDrive/Fetch Sample Data/receipts.json') as f:
    lines = f.readlines()

# Normalize each JSON object and convert date columns
normalized_data = []
for line in lines:
    data = json.loads(line)
    df_receipts = pd.json_normalize(data)

    # Convert specific date columns to datetime format
    date_columns = ['createDate.$date', 'dateScanned.$date', 'finishedDate.$date', 'modifyDate.$date', 'pointsAwardedDate.$date', 'purchaseDate.$date']
    for column in date_columns:
        if column in df_receipts.columns:
            df_receipts[column] = pd.to_datetime(df_receipts[column]/1000, unit='s')

    normalized_data.append(df_receipts)

# Concatenate all normalized DataFrames
df_receipts = pd.concat(normalized_data, ignore_index=True)

# Print the resulting DataFrame
df_receipts.head(5)


,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


Need to further normalize the rewardsReceiptItemList column

In [49]:
df_receipts = df_receipts.explode('rewardsReceiptItemList')
df_receipts.reset_index(inplace=True)

In [50]:
df_receipts = df_receipts.fillna({'rewardsReceiptItemList':'{}'})
df_receipts['rewardsReceiptItemList'] = df_receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
df_receipts['rewardsReceiptItemList'] = df_receipts['rewardsReceiptItemList'].apply(literal_eval)

In [45]:
df_receipts.head(5)

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


In [54]:
df_receipts_norm = pd.json_normalize(df_receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')

df_receipts_norm.head(5)

,rewardsReceiptItemList.barcode,rewardsReceiptItemList.description,rewardsReceiptItemList.finalPrice,rewardsReceiptItemList.itemPrice,rewardsReceiptItemList.needsFetchReview,rewardsReceiptItemList.partnerItemId,rewardsReceiptItemList.preventTargetGapPoints,rewardsReceiptItemList.quantityPurchased,rewardsReceiptItemList.userFlaggedBarcode,rewardsReceiptItemList.userFlaggedNewItem,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Merging the exploded column back to the original receipts dataframe

In [53]:
df_receipts_final = pd.merge(df_receipts, df_receipts_norm, left_index=True, right_index=True, how='outer')
df_receipts_final.head(5)

,index,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5.0,All-receipts receipt bonus,5,1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5.0,All-receipts receipt bonus,5.0,4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
#User friendly names for the colum headers

cols = {'_id.$oid':'receipt_id',
        'createDate.$date':'createDate',
        'dateScanned.$date':'dateScanned',
        'finishedDate.$date':'finishedDate',
        'modifyDate.$date':'modifyDate',
        'pointsAwardedDate.$date':'pointsAwardedDate',
        'purchaseDate.$date':'purchaseDate'}

df_receipts_final.rename(columns=cols,
          inplace=True)

df_receipts_final.columns

Index(['index', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'pointsEarned',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId', 'receipt_id', 'createDate', 'dateScanned',
       'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate',
       'rewardsReceiptItemList.barcode', 'rewardsReceiptItemList.description',
       'rewardsReceiptItemList.finalPrice', 'rewardsReceiptItemList.itemPrice',
       'rewardsReceiptItemList.needsFetchReview',
       'rewardsReceiptItemList.partnerItemId',
       'rewardsReceiptItemList.preventTargetGapPoints',
       'rewardsReceiptItemList.quantityPurchased',
       'rewardsReceiptItemList.userFlaggedBarcode',
       'rewardsReceiptItemList.userFlaggedNewItem',
       'rewardsReceiptItemList.userFlaggedPrice',
       'rewardsReceiptItemList.userFlaggedQuantity',
       'rewardsReceiptItemList.needsFetchReviewReason',
       'rewardsReceiptItemList.pointsNotAwardedReason',
       'rew

Normalizing and reading the Brands table

In [30]:
# Read the JSON file with one JSON object per line
with open('/content/drive/MyDrive/Fetch Sample Data/brands.json') as f:
    lines = f.readlines()

# Normalize each JSON object and convert date columns
normalized_data = []
for line in lines:
    data = json.loads(line)
    df_brands = pd.json_normalize(data)
    normalized_data.append(df_brands)

# Concatenate all normalized DataFrames
df_brands = pd.concat(normalized_data, ignore_index=True)

# Print the resulting DataFrame
df_brands.head(5)


,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [57]:
#Renaming column headers to user-friendly names

brandcols = {'_id.$oid':'brand_id',
             'cpg.$id.$oid':'cpg_id',
             'cpg.$ref':'cpf_ref'}

df_brands.rename(columns= brandcols, inplace=True)

df_brands.columns

Index(['barcode', 'category', 'categoryCode', 'name', 'topBrand', 'brand_id',
       'cpg_id', 'cpf_ref', 'brandCode'],
      dtype='object')

Normalizing and reading users table


In [40]:
# Read the JSON file with one JSON object per line
with open('/content/drive/MyDrive/Fetch Sample Data/users.json') as f:
    lines = f.readlines()

# Normalize each JSON object and convert date columns
normalized_data = []
for line in lines:
    data = json.loads(line)
    df_users = pd.json_normalize(data)

    # Convert specific date columns to datetime format
    date_columns = ['createdDate.$date', 'lastLogin.$date']
    for column in date_columns:
        if column in df_users.columns:
            df_users[column] = pd.to_datetime(int(df_users[column])/1000, unit='s')

    normalized_data.append(df_users)

# Concatenate all normalized DataFrames
df_users = pd.concat(normalized_data, ignore_index=True)

# Print the resulting DataFrame
df_users.head(5)

,active,role,signUpSource,state,_id.$oid,createdDate.$date,lastLogin.$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.553999872,2021-01-03 15:25:30.596999936
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000000,2021-01-03 15:25:37.858000128


In [59]:
#Renaming column headers to user-friendly names

usercols = {'_id.$oid':'user_id',
             'createdDate.$date':'createdDate',
             'lastLogin.$date':'lastLogin'}

df_users.rename(columns= usercols, inplace=True)

df_users.columns

Index(['active', 'role', 'signUpSource', 'state', 'brand_id', 'createdDate',
       'lastLogin'],
      dtype='object')

# Q2. Writing a SQL Query that directly answers the Stakeholders question